In [1]:
import os, random, logging
import utils.auxua as aux
import utils.watch as watch
import numpy as np

''' tf log level & neptune'''
aux.set_tf_loglevel(logging.ERROR)

Num GPUs Available:  4
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')
PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')
PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')


/home/jtstudents/anaconda3/envs/zugpu/lib/python3.8/site-packages/neptune/internal/backends/hosted_client.py:48: NeptuneDeprecationWarning: The 'neptune-client' package has been deprecated and will be removed in the future. Install the 'neptune' package instead. For more, see https://docs.neptune.ai/setup/upgrading/
  from neptune.version import version as neptune_client_version


In [2]:
'''get_results_from_txt'''

res_list_full,res_list_max,res_list_fn,res_list_path,res_list_labels,res_path,res_model_fn,res_model_fn_strap =\
aux.get_results_from_txt(aux.RSLT_PATH,printt=True)

res_list={
    'full':res_list_full,
    'max':res_list_max,
    'videoname':res_list_fn,
    'videopath':res_list_path,
    'labels':res_list_labels,
    'txt_path':res_path,
    'txt_modelname':res_model_fn,
    'txt_modelname_strap':res_model_fn_strap        
}

for i in range(len(res_list['txt_modelname'])):print(i,res_list['txt_modelname'][i])

#res_list_1 = aux.get_results_from_txt("/raid/DATASETS/.zuble/vigia/zu++/model/rslt/1671124367.446378_3gelu_adamamsgrad_0_4000_weights.txt")

FOLDER

results for 3gelu_sgd_0_4000_weights
	ACCURACY   0.81875
	PRECISION (%% of True 1 out of all Positive predicted)  0.8867102
	RECALL (%% of True Positive out of all actual anomalies)  0.814
	AUPRC ( AreaUnderPrecisionRecallCurve ) 0.9295 
	 AUC-ROC 0.8896 
	F1_SCORE (harmonic mean of precision and recall)  0.8488007709181216

results for 3gelu_adam_0_4000_weights
	ACCURACY   0.77125
	PRECISION (%% of True 1 out of all Positive predicted)  0.9116883
	RECALL (%% of True Positive out of all actual anomalies)  0.702
	AUPRC ( AreaUnderPrecisionRecallCurve ) 0.9290 
	 AUC-ROC 0.8835 
	F1_SCORE (harmonic mean of precision and recall)  0.7932203158709636

results for 3gelu_adamamsgrad_0_4000_weights
	ACCURACY   0.7875
	PRECISION (%% of True 1 out of all Positive predicted)  0.9166667
	RECALL (%% of True Positive out of all actual anomalies)  0.726
	AUPRC ( AreaUnderPrecisionRecallCurve ) 0.9398 
	 AUC-ROC 0.9044 
	F1_SCORE (harmonic mean of precision and recall)  0.8102678471795525

res

In [3]:
def init_watch(res_list, txt_i,watch_this=''):
    
    def get_fn_index_from_list(txt_i):
        '''retrives all video indexs from a txt that are false negatives'''
        
        print("\nget_fn_index_from_list\n\tfn indexs from",res_list['txt_path'][txt_i])
        indexs = []
        labels_count={'B1fitgh':0,'B2shoot':0,'B4riot':0,'B5abuse':0,'B6caracc':0,'Gexplosao':0}
        for video_j in range(len(res_list['videoname'])):
            if 'label_A' not in res_list['videoname'][video_j][txt_i]:
                if float(res_list['max'][video_j][txt_i])<=0.5: 
                    indexs.append(video_j)
                    if 'label_B1' in res_list['videoname'][video_j][txt_i]:labels_count['B1fitgh']+=1
                    if 'label_B2' in res_list['videoname'][video_j][txt_i]:labels_count['B2shoot']+=1
                    if 'label_B4' in res_list['videoname'][video_j][txt_i]:labels_count['B4riot']+=1
                    if 'label_B5' in res_list['videoname'][video_j][txt_i]:labels_count['B5abuse']+=1
                    if 'label_B6' in res_list['videoname'][video_j][txt_i]:labels_count['B6caracc']+=1
                    if 'label_G' in res_list['videoname'][video_j][txt_i]:labels_count['Gexplosao']+=1    
        
        print("\t",len(indexs)," false negatives indexs:\n\t",indexs)
        print(labels_count)
        return indexs
    
    as_total = watch.get_as_total_from_res_list(res_list,txt_i)
    
    if watch_this == 'fn':
        watch_this_videos = get_fn_index_from_list(txt_i)
        for video_j in watch_this_videos:
            watch.cv2_test_from_aslist(res_list,as_total[video_j],video_j,txt_i)
    else:
        for video_j in range(len(res_list['videopath'])):
            watch.cv2_test_from_aslist(res_list,as_total[video_j],video_j,txt_i)

In [4]:
init_watch(res_list,8,'fn')

as_total from /raid/DATASETS/.zuble/vigia/zu++/model/rslt/1677498953.2416248_relu_sgd_0_4000_weights-2_500.txt

get_fn_index_from_list
	fn indexs from /raid/DATASETS/.zuble/vigia/zu++/model/rslt/1677498953.2416248_relu_sgd_0_4000_weights-2_500.txt
	 2  false negatives indexs:
	 [631, 780]
{'B1fitgh': 0, 'B2shoot': 1, 'B4riot': 0, 'B5abuse': 0, 'B6caracc': 0, 'Gexplosao': 1}

res_list['full'] for video  631 | txt 8 ['2', '501', '0.4965662', '500', '624', '0.46423677']

res_list['full'] for video  780 | txt 8 ['3', '501', '0.18835111', '500', '1001', '0.19460565', '1000', '1081', '0.1977591']
